<a href="https://colab.research.google.com/github/dave20874/RapidReact-Trajectory/blob/main/RapidReact_Trajectory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import math
import numpy as np

import matplotlib.pyplot as plt
import numpy as np

In [9]:
x = np.divide([2.4, 2.5], 2.0)
print(x)

[1.2  1.25]


In [41]:
in_to_m = 25.4/1000
flywheel_r = 2.0 * in_to_m
high_goal_h = (8*12+8) * in_to_m  # 8' 8" Goal
low_goal_h = (3*12+5) * in_to_m   # 3' 5" Goal
shooter_h = 16.0 * in_to_m        # 16" height of shooter
gravity = 9.8                     # m/s^2
m_to_ft = 1000.0/25.4/12.0
ft_to_m = 1.0/m_to_ft
shooter_loss = 0.575
CRITICAL_ANGLE = math.radians(-30.0)
MAX_HEIGHT = 21.0 * ft_to_m
m_ball_kg = 0.268
spin_hz = -589.0/60.0   # Shooter constant
ball_r = 9.5/2.0*in_to_m
ball_a = math.pi*ball_r*ball_r
rho = 1.225    # kg / m^3

# Extra height so ball clears the target by 6 inches.
clearance = (ball_r + 6.0) * in_to_m


def launch_v(rpm):
  v_mps = rpm/60.0*flywheel_r*2.0*math.pi  # rev/min / sec/min * m -> m/sec
  return v_mps*shooter_loss

def normalize(vector):
  mag = math.sqrt(vector[0]*vector[0] + vector[1]*vector[1])
  return np.divide(vector, mag)

# Based on https://www1.grc.nasa.gov/beginners-guide-to-aeronautics/ideal-lift-of-a-spinning-ball/
def lift(v, spin=-spin_hz):
  pi_2 = math.pi * math.pi
  b_3 = ball_r * ball_r * ball_r

  L = 4.0 / 3.0 * (4.0 * pi_2 * b_3 * spin * rho * v)
  return L


# Returns None if shot is invalid
# Returns (dist, arr_angle, max_height, rpm, trajectory) if shot is valid
def shoot(rpm, angle_rad, goal_h=high_goal_h):
  v0 = launch_v(rpm)
  # print(f"v0 = {v0}")
  vx = v0 * math.cos(angle_rad)
  vy = v0 * math.sin(angle_rad)

  x = 0.0
  y = shooter_h

  state = [x, y, vx, vy]

  h = 0.001     # 1 ms steps

  # Advance trajectory 1ms at a time until it is descending below goal height
  trajectory = []
  trajectory.append((x, y))
  report_interval = 10
  h_max = y
  cd = 0.4  # coefficient of drag

  # While traveling up or above goal height
  while (state[3] > 0.0) | (state[1] > goal_h):

    # velocity
    v = [state[2], state[3]]
    unit_v = normalize(v)
    q = 0.5 * rho * (v[0]*v[0] + v[1]*v[1])

    perp_v = [-v[1], v[0]]
    unit_perp_v = [-unit_v[1], unit_v[0]]

    f_gravity = np.multiply([0.0, -gravity], m_ball_kg)
    f_drag = np.multiply(unit_v, -cd*ball_a*q)  # TODO

    L = lift(-math.sqrt(v[0]*v[0]+v[1]*v[1]))
    f_magnus = np.multiply(unit_perp_v, L) # TODO
    # f_magnus = [0.0, 0.0]  # TODO; Fix lift / magnus.
    f_total = np.add.reduce([f_gravity, f_drag, f_magnus])
    print(f"f_total: {f_total}, f_magnus: {f_magnus}")
    accel = np.divide(f_total, m_ball_kg)

    # Derivative of pos is vel
    # Derivative of vel is accel
    state_dot = [state[2], state[3], accel[0], accel[1]]

    state += np.multiply(state_dot, h)

    # v_new = vy - h*gravity
    # x += vx*h
    # y += (v_new+vy)/2.0 * h
    # vy = v_new

    if state[1] > h_max:
      h_max = state[1]

    report_interval -= 1
    if report_interval == 0:
      trajectory.append((state[0], state[1]))
      report_interval = 10

  arr_angle = math.atan2(state[3], state[2])

  retval = (state[0], arr_angle, h_max, rpm, trajectory)

  if h_max < goal_h + clearance:
    # print("Too low")
    return None

  if h_max >= MAX_HEIGHT:
    # print("Too high")
    return None

  if arr_angle > CRITICAL_ANGLE:
    # print("Too shallow")
    return None

  return retval

result = shoot(3000.0, math.radians(80.0))

print(f"dist:{result[0]*m_to_ft} ft, arr_ang:{math.degrees(result[1])} deg, max_h:{result[2]*m_to_ft} ft")

f_total: [ 9.88263571 -5.32701172], f_magnus: [10.04646959 -1.77146365]
f_total: [ 9.85698662 -5.36463993], f_magnus: [10.02437278 -1.81245749]
f_total: [ 9.83120173 -5.40215496], f_magnus: [10.0021199  -1.85334493]
f_total: [ 9.80528151 -5.43955635], f_magnus: [ 9.97971139 -1.89412542]
f_total: [ 9.77922642 -5.47684364], f_magnus: [ 9.95714775 -1.93479838]
f_total: [ 9.7530369  -5.51401639], f_magnus: [ 9.93442943 -1.97536327]
f_total: [ 9.72671342 -5.55107414], f_magnus: [ 9.91155692 -2.01581953]
f_total: [ 9.70025643 -5.58801646], f_magnus: [ 9.88853069 -2.05616659]
f_total: [ 9.67366641 -5.62484288], f_magnus: [ 9.86535122 -2.0964039 ]
f_total: [ 9.6469438  -5.66155297], f_magnus: [ 9.842019   -2.13653092]
f_total: [ 9.62008907 -5.69814629], f_magnus: [ 9.81853449 -2.17654709]
f_total: [ 9.5931027  -5.73462238], f_magnus: [ 9.7948982  -2.21645187]
f_total: [ 9.56598513 -5.77098083], f_magnus: [ 9.7711106  -2.25624471]
f_total: [ 9.53873684 -5.80722117], f_magnus: [ 9.74717218 -2.29

TypeError: ignored

In [42]:
test_angle = 70

for angle_deg in range(test_angle, test_angle+1):
  angle = math.radians(angle_deg)

  min_range = None
  max_range = None
  for rpm in range(2000, 4500, 10):
    soln = shoot(rpm, angle)
    if soln is not None:
      if (min_range is None):
        min_range = soln
      elif (soln[0] < min_range[0]):
        # replace min_range solution with this solution
        min_range = soln

      if (max_range is None):
        max_range = soln
      elif (soln[0] > max_range[0]):
        # replace max_range solution with this solution
        max_range = soln

  # report min, max range solutions
  print(f"Angle: {angle_deg} deg:")
  print(f"  min: {min_range[3]} rpm, {min_range[0]*m_to_ft} ft, arr: {math.degrees(min_range[1])} deg, max_h: {min_range[2]*m_to_ft} ft")
  print(f"  max: {max_range[3]} rpm, {max_range[0]*m_to_ft} ft, arr: {math.degrees(max_range[1])} deg, max_h: {max_range[2]*m_to_ft} ft")

  # Form x, y values in feet.
  min_x = []
  min_y = []
  for point in min_range[4]:
    x = (point[0]-min_range[0])*m_to_ft
    y = (point[1])*m_to_ft
    min_x.append(x)
    min_y.append(y)
    # print(f"{x}, {y}")

  max_x = []
  max_y = []
  for point in max_range[4]:
    x = (point[0]-max_range[0])*m_to_ft
    y = (point[1])*m_to_ft
    max_x.append(x)
    max_y.append(y)

  # plot
  fig, ax = plt.subplots()

  ax.plot(min_x, min_y, linewidth=2.0)
  ax.plot(max_x, max_y, linewidth=2.0)

  ax.set(xlim=(-30, 2), xticks=np.arange(-30, 2),
         ylim=(0, 30), yticks=np.arange(0, 30))

  plt.show()

Streaming output truncated to the last 5000 lines.
f_total: [  7.71871795 -13.16311284], f_magnus: [ 8.83566353 -9.49761452]
f_total: [  7.66152618 -13.18762654], f_magnus: [ 8.78106205 -9.52963228]
f_total: [  7.604264   -13.21190861], f_magnus: [ 8.72635889 -9.56141281]
f_total: [  7.54693227 -13.23595884], f_magnus: [ 8.671555   -9.59295581]
f_total: [  7.48953183 -13.25977701], f_magnus: [ 8.61665135 -9.62426099]
f_total: [  7.43206355 -13.28336294], f_magnus: [ 8.56164891 -9.65532807]
f_total: [  7.37452827 -13.30671641], f_magnus: [ 8.50654862 -9.68615677]
f_total: [  7.31692686 -13.32983724], f_magnus: [ 8.45135147 -9.71674681]
f_total: [  7.25926017 -13.35272522], f_magnus: [ 8.39605841 -9.74709792]
f_total: [  7.20152905 -13.37538017], f_magnus: [ 8.3406704  -9.77720982]
f_total: [  7.14373436 -13.39780188], f_magnus: [ 8.28518843 -9.80708224]
f_total: [  7.08587697 -13.41999019], f_magnus: [ 8.22961344 -9.83671493]
f_total: [  7.02795772 -13.44194489], f_magnus: [ 8.17394642 

TypeError: ignored